# Data Integration over price & volume data and fundamental data

Below I import price and volume data as well as the fundamental data from exsiting dataset. I then do:
* Drop text data
* Merge the fundamental data to price and volume data, so that every day we should expect data for price, volume and fundamental data
* Impute missing data in merged daily fundamental data in the following appetite:
    * If the current day's data is missing, impute this day's data with the most recent valid data
    * Impute missing data before the first record days with 0 (after all we truly cannot see any data at this time), the statistics has shown that such 0 takes a very trival proportion of all the data
* Such imputation strategy can be justified as such that while we acknowledge fundamental data cannot be expected to be updated publicly on a daily basis, it is still a great resource for us to make decision during daily investment so we still need to look at it on a daily basis, and thus if there is no updated fundamental data available this day we can only refer to the most recent fundamental data as a current status of the given ticker.

|  Feature  |  Availble?  | Can Calculate? |
|  ----  | ----  | ----  |
| Accruals | No | Yes |
| Abnormal Earning Announcement Volume | Yes | Yes |
| Asset Growth | No | Yes |
| Bid-ask spread | No | No |
| Beta | No | Yes |
| Book To Market | No | Yes |
| Cash Holdings | Yes | No |
| Cash Flow to Debt | No | Yes |
| Cash Flow to Price | No | Yes |
| Change in Inventory | No | Yes |
| Change in 6-month momentum | No | Yes |
| Change in tax | No | Yes |
| Investments | No | Yes |
| Depreciation | Yes | No |
| Dividend to Price | No | Yes |
| Growth in Common Stock | No | Yes |
| Earnings to Price | No | Yes |
| Gross Profit | Yes | No |
| Capital Expenditures | Yes | No |
| Net Operating Assets | Yes | No |
| Leverage | No | Yes |
| Growth in long-term debt | No | Yes |
| 12 Month Momentum | No | Yes |
| 1 Month Momentum | No | Yes |
| 36 Month Momentum | No | Yes |
| 6 Month Momentum | No | Yes |
| Change in Deprecation | No | Yes |
| Return volatility | No | Yes |
| Return on Assets | No | Yes |
| Earning Volatility| No | Yes |
| Return on Equity | No | Yes |
| Return on Capital | No | Yes |
| Revenue Surprise | No | Yes |
| Income growth | No | Yes |
| Cash Flow Volatility | No | Yes |


In [1]:
import pandas as pd

f = pd.read_csv("fundamental.csv", index_col=0)
f

,currency_symbol,totalAssets,intangibleAssets,earningAssets,otherCurrentAssets,totalLiab,totalStockholderEquity,deferredLongTermLiab,otherCurrentLiab,commonStock,...,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares,preferredStockAndOtherAdjustments,beforeAfterMarket,currency,epsActual,epsEstimate,epsDifference,surprisePercent,Ticker
1985-09-30,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
1985-12-31,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
1986-03-31,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
1986-06-30,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
1986-09-30,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30,USD,1.377000e+10,1.390000e+09,NaN,507000000.0,9.190000e+09,4.580000e+09,274000000.0,1.269000e+09,5000000.0,...,529000000.0,529000000.0,NaN,BeforeMarket,USD,1.2,1.22,-0.02,-1.6393,ZTS
2022-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,BeforeMarket,USD,NaN,1.25,NaN,NaN,ZTS
2022-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,AfterMarket,USD,NaN,NaN,NaN,NaN,ZTS
2023-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,BeforeMarket,USD,NaN,NaN,NaN,NaN,ZTS


In [2]:
cols = f.columns
for col in cols[:-1]:
    if f[col].dtype == float or f[col].dtype == int:
        continue
    else:
        print("ignore " + col)
        f = f.drop(col, axis = 1)

ignore currency_symbol
ignore beforeAfterMarket
ignore currency


In [3]:
f = f.reset_index().rename(columns={'index': 'Date'}).reset_index()
f

,index,Date,totalAssets,intangibleAssets,earningAssets,otherCurrentAssets,totalLiab,totalStockholderEquity,deferredLongTermLiab,otherCurrentLiab,...,totalOtherIncomeExpenseNet,discontinuedOperations,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares,preferredStockAndOtherAdjustments,epsActual,epsEstimate,epsDifference,surprisePercent,Ticker
0,0,1985-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
1,1,1985-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2,2,1986-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
3,3,1986-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
4,4,1986-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62114,62114,2022-06-30,1.377000e+10,1.390000e+09,NaN,507000000.0,9.190000e+09,4.580000e+09,274000000.0,1.269000e+09,...,-56000000.0,NaN,529000000.0,529000000.0,NaN,1.2,1.22,-0.02,-1.6393,ZTS
62115,62115,2022-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.25,NaN,NaN,ZTS
62116,62116,2022-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZTS
62117,62117,2023-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZTS


In [4]:
first_indexes = f.groupby('Ticker')['index'].min()
cols = [i for i in f.columns if (i != 'index' and i != 'Date')]
f.loc[first_indexes, cols] = f.loc[first_indexes, cols].fillna(0)
f = f.fillna(method="ffill")
f

,index,Date,totalAssets,intangibleAssets,earningAssets,otherCurrentAssets,totalLiab,totalStockholderEquity,deferredLongTermLiab,otherCurrentLiab,...,totalOtherIncomeExpenseNet,discontinuedOperations,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares,preferredStockAndOtherAdjustments,epsActual,epsEstimate,epsDifference,surprisePercent,Ticker
0,0,1985-09-30,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0000,MMM
1,1,1985-12-31,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0000,MMM
2,2,1986-03-31,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0000,MMM
3,3,1986-06-30,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0000,MMM
4,4,1986-09-30,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0000,MMM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62114,62114,2022-06-30,1.377000e+10,1.390000e+09,0.0,507000000.0,9.190000e+09,4.580000e+09,274000000.0,1.269000e+09,...,-56000000.0,0.0,529000000.0,529000000.0,0.0,1.2,1.22,-0.02,-1.6393,ZTS
62115,62115,2022-09-30,1.377000e+10,1.390000e+09,0.0,507000000.0,9.190000e+09,4.580000e+09,274000000.0,1.269000e+09,...,-56000000.0,0.0,529000000.0,529000000.0,0.0,1.2,1.25,-0.02,-1.6393,ZTS
62116,62116,2022-12-31,1.377000e+10,1.390000e+09,0.0,507000000.0,9.190000e+09,4.580000e+09,274000000.0,1.269000e+09,...,-56000000.0,0.0,529000000.0,529000000.0,0.0,1.2,1.25,-0.02,-1.6393,ZTS
62117,62117,2023-03-31,1.377000e+10,1.390000e+09,0.0,507000000.0,9.190000e+09,4.580000e+09,274000000.0,1.269000e+09,...,-56000000.0,0.0,529000000.0,529000000.0,0.0,1.2,1.25,-0.02,-1.6393,ZTS


In [5]:
import numpy as np

f['Accural'] = 0
f['Accrual'] = f['earningAssets'].diff()
f.loc[first_indexes, 'Accrual'] = 0

f['assetGrowth'] = 0
f['assetGrowth'] = f['totalAssets'].pct_change().replace([np.inf, -np.inf], np.nan).fillna(0)
f.loc[first_indexes, 'assetGrowth'] = 0
    
f['cashToDebt'] = (f['cash'] / f['netDebt']).replace([np.inf, -np.inf], np.nan).fillna(0)

f['inventoryGrowth'] = 0
f['inventoryGrowth'] = f['inventory'].pct_change().replace([np.inf, -np.inf], np.nan).fillna(0)
f.loc[first_indexes, 'inventoryGrowth'] = 0

f['taxGrowth'] = 0
f['taxGrowth'] = f['taxProvision'].pct_change().replace([np.inf, -np.inf], np.nan).fillna(0)
f.loc[first_indexes, 'taxGrowth'] = 0

f['commonStockGrowth'] = 0
f['commonStockGrowth'] = f['commonStock'].pct_change().replace([np.inf, -np.inf], np.nan).fillna(0)
f.loc[first_indexes, 'commonStockGrowth'] = 0

f['leverage'] = (f['netDebt'] / f['ebitda']).replace([np.inf, -np.inf], np.nan).fillna(0)

f['longTermDebtGrowth'] = 0
f['longTermDebtGrowth'] = f['longTermDebt'].pct_change().replace([np.inf, -np.inf], np.nan).fillna(0)
f.loc[first_indexes, 'longTermDebtGrowth'] = 0

f['depreciationGrowth'] = 0
f['depreciationGrowth'] = f['depreciation'].pct_change().replace([np.inf, -np.inf], np.nan).fillna(0)
f.loc[first_indexes, 'depreciationGrowth'] = 0

f['equityGrowth'] = 0
f['equityGrowth'] = f['totalStockholderEquity'].pct_change().replace([np.inf, -np.inf], np.nan).fillna(0)
f.loc[first_indexes, 'equityGrowth'] = 0

f

,index,Date,totalAssets,intangibleAssets,earningAssets,otherCurrentAssets,totalLiab,totalStockholderEquity,deferredLongTermLiab,otherCurrentLiab,...,Accrual,assetGrowth,cashToDebt,inventoryGrowth,taxGrowth,commonStockGrowth,leverage,longTermDebtGrowth,depreciationGrowth,equityGrowth
0,0,1985-09-30,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
1,1,1985-12-31,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.000000,0.000000,0.000000,0.00000,0.0,-0.000000,0.000000,0.000000,0.000000
2,2,1986-03-31,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
3,3,1986-06-30,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
4,4,1986-09-30,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62114,62114,2022-06-30,1.377000e+10,1.390000e+09,0.0,507000000.0,9.190000e+09,4.580000e+09,274000000.0,1.269000e+09,...,0.0,-0.006494,0.648067,0.071949,0.06015,0.0,4.864286,-0.001339,0.026316,-0.016745
62115,62115,2022-09-30,1.377000e+10,1.390000e+09,0.0,507000000.0,9.190000e+09,4.580000e+09,274000000.0,1.269000e+09,...,0.0,0.000000,0.648067,0.000000,0.00000,0.0,4.864286,0.000000,0.000000,0.000000
62116,62116,2022-12-31,1.377000e+10,1.390000e+09,0.0,507000000.0,9.190000e+09,4.580000e+09,274000000.0,1.269000e+09,...,0.0,0.000000,0.648067,0.000000,0.00000,0.0,4.864286,0.000000,0.000000,0.000000
62117,62117,2023-03-31,1.377000e+10,1.390000e+09,0.0,507000000.0,9.190000e+09,4.580000e+09,274000000.0,1.269000e+09,...,0.0,0.000000,0.648067,0.000000,0.00000,0.0,4.864286,0.000000,0.000000,0.000000


In [6]:
d = pd.read_csv("price.csv")
m = pd.merge(d, f, left_on=['Date', 'Ticker'], right_on=['Date', 'Ticker'], how = 'left')
m

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,...,Accrual,assetGrowth,cashToDebt,inventoryGrowth,taxGrowth,commonStockGrowth,leverage,longTermDebtGrowth,depreciationGrowth,equityGrowth
0,0,1999-11-18 00:00:00-05:00,27.847825,30.602008,24.481605,26.929766,62546380.0,0.000,0.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1999-11-19 00:00:00-05:00,26.279474,26.317727,24.366850,24.711123,15234146.0,0.000,0.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1999-11-22 00:00:00-05:00,25.284907,26.929766,24.519857,26.929766,6577870.0,0.000,0.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1999-11-23 00:00:00-05:00,26.011702,26.700248,24.481602,24.481602,5975611.0,0.000,0.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1999-11-24 00:00:00-05:00,24.558100,25.667423,24.481594,25.131887,4843231.0,0.000,0.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4122979,4122979,2022-10-31 00:00:00-04:00,152.110001,153.339996,149.839996,150.779999,1899000.0,0.325,0.0,ZTS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4122980,4122980,2022-11-01 00:00:00-04:00,151.399994,154.910004,150.380005,152.720001,2286200.0,0.000,0.0,ZTS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4122981,4122981,2022-11-02 00:00:00-04:00,152.869995,153.210007,147.190002,147.360001,2638300.0,0.000,0.0,ZTS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4122982,4122982,2022-11-03 00:00:00-04:00,134.149994,134.149994,124.150002,131.139999,8065500.0,0.000,0.0,ZTS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
t = m[['Ticker']]
t['index'] = m.index
first_record_index = t.groupby('Ticker')['index'].min().values
m.loc[first_record_index, :] = m.loc[first_record_index, :].fillna(0)
m = m.fillna(method="ffill")
m

/var/folders/5f/54m_k2jx639_v5w_34k81lg40000gn/T/ipykernel_1935/2395876625.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['index'] = m.index


,Unnamed: 0,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,...,Accrual,assetGrowth,cashToDebt,inventoryGrowth,taxGrowth,commonStockGrowth,leverage,longTermDebtGrowth,depreciationGrowth,equityGrowth
0,0,1999-11-18 00:00:00-05:00,27.847825,30.602008,24.481605,26.929766,62546380.0,0.000,0.0,A,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1999-11-19 00:00:00-05:00,26.279474,26.317727,24.366850,24.711123,15234146.0,0.000,0.0,A,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1999-11-22 00:00:00-05:00,25.284907,26.929766,24.519857,26.929766,6577870.0,0.000,0.0,A,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1999-11-23 00:00:00-05:00,26.011702,26.700248,24.481602,24.481602,5975611.0,0.000,0.0,A,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1999-11-24 00:00:00-05:00,24.558100,25.667423,24.481594,25.131887,4843231.0,0.000,0.0,A,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4122979,4122979,2022-10-31 00:00:00-04:00,152.110001,153.339996,149.839996,150.779999,1899000.0,0.325,0.0,ZTS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4122980,4122980,2022-11-01 00:00:00-04:00,151.399994,154.910004,150.380005,152.720001,2286200.0,0.000,0.0,ZTS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4122981,4122981,2022-11-02 00:00:00-04:00,152.869995,153.210007,147.190002,147.360001,2638300.0,0.000,0.0,ZTS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4122982,4122982,2022-11-03 00:00:00-04:00,134.149994,134.149994,124.150002,131.139999,8065500.0,0.000,0.0,ZTS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
m['Book To Market'] = ((m['totalAssets'] - m['totalLiab']) / (m['Close'] * m['Volume'])).replace([np.inf, -np.inf], np.nan).fillna(0)
m['Cash To Price'] = m['cash'] / m['Close']
m['Dividend To Price'] = m['Dividends'] / m['Close']
m['Earning To Price'] = m['totalRevenue'] / m['Close']


m['30 Day Momentum'] = m['Close'].diff(periods=30)
first_30_indexes = []
for i in first_indexes:
    for j in range(30):
        first_30_indexes.append(i + j)
        
m.loc[first_30_indexes, '30 Day Momentum'] = 0

m['180 Day Momentum'] = m['Close'].diff(periods=180)
first_180_indexes = []
for i in first_indexes:
    for j in range(180):
        first_180_indexes.append(i + j)
        
m.loc[first_180_indexes, '180 Day Momentum'] = 0

m['360 Day Momentum'] = m['Close'].diff(periods=360)
first_360_indexes = []
for i in first_indexes:
    for j in range(360):
        first_360_indexes.append(i + j)
        
m.loc[first_360_indexes, '360 Day Momentum'] = 0

m['1080 Day Momentum'] = m['Close'].diff(periods=1080)
first_1080_indexes = []
for i in first_indexes:
    for j in range(1080):
        first_1080_indexes.append(i + j)
        
m.loc[first_1080_indexes, '1080 Day Momentum'] = 0

m['return'] = m['Close'].pct_change()
m['er'] = m['return'] - m['market return']
m['er'] = m['er'].shift(-1)
m.loc[[i - 1 for i in first_indexes if (i - 1 > 0)], 'er'] = np.nan
m = m.dropna()
m

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,...,Book To Market,Cash To Price,Dividend To Price,Earning To Price,30 Day Momentum,180 Day Momentum,360 Day Momentum,1080 Day Momentum,return,er
1,1,1999-11-19 00:00:00-05:00,26.279474,26.317727,24.366850,24.711123,15234146.0,0.000,0.0,A,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.082386,0.090529
2,2,1999-11-22 00:00:00-05:00,25.284907,26.929766,24.519857,26.929766,6577870.0,0.000,0.0,A,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.089783,-0.079438
3,3,1999-11-23 00:00:00-05:00,26.011702,26.700248,24.481602,24.481602,5975611.0,0.000,0.0,A,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.090909,0.017706
4,4,1999-11-24 00:00:00-05:00,24.558100,25.667423,24.481594,25.131887,4843231.0,0.000,0.0,A,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.026562,0.003369
5,5,1999-11-26 00:00:00-05:00,25.017140,25.399665,24.940636,25.208403,1729466.0,0.000,0.0,A,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.003045,0.028967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4122978,4122978,2022-10-28 00:00:00-04:00,151.777511,153.553736,150.200859,152.955002,1615500.0,0.000,0.0,ZTS,...,0.0,0.0,0.000000,0.0,-4.011475,-45.280731,-20.660767,69.528549,0.014159,-0.006765
4122979,4122979,2022-10-31 00:00:00-04:00,152.110001,153.339996,149.839996,150.779999,1899000.0,0.325,0.0,ZTS,...,0.0,0.0,0.002155,0.0,-6.076706,-46.958588,-22.192078,67.178452,-0.014220,0.016968
4122980,4122980,2022-11-01 00:00:00-04:00,151.399994,154.910004,150.380005,152.720001,2286200.0,0.000,0.0,ZTS,...,0.0,0.0,0.000000,0.0,0.104279,-42.592453,-22.242554,69.512009,0.012866,-0.010095
4122981,4122981,2022-11-02 00:00:00-04:00,152.869995,153.210007,147.190002,147.360001,2638300.0,0.000,0.0,ZTS,...,0.0,0.0,0.000000,0.0,-3.559326,-49.274902,-25.136749,64.278610,-0.035097,-0.099485


In [9]:
m = m.drop('Unnamed: 0', axis=1)
m

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Adj Close,...,Book To Market,Cash To Price,Dividend To Price,Earning To Price,30 Day Momentum,180 Day Momentum,360 Day Momentum,1080 Day Momentum,return,er
1,1999-11-19 00:00:00-05:00,26.279474,26.317727,24.366850,24.711123,15234146.0,0.000,0.0,A,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.082386,0.090529
2,1999-11-22 00:00:00-05:00,25.284907,26.929766,24.519857,26.929766,6577870.0,0.000,0.0,A,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.089783,-0.079438
3,1999-11-23 00:00:00-05:00,26.011702,26.700248,24.481602,24.481602,5975611.0,0.000,0.0,A,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.090909,0.017706
4,1999-11-24 00:00:00-05:00,24.558100,25.667423,24.481594,25.131887,4843231.0,0.000,0.0,A,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.026562,0.003369
5,1999-11-26 00:00:00-05:00,25.017140,25.399665,24.940636,25.208403,1729466.0,0.000,0.0,A,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.003045,0.028967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4122978,2022-10-28 00:00:00-04:00,151.777511,153.553736,150.200859,152.955002,1615500.0,0.000,0.0,ZTS,0.0,...,0.0,0.0,0.000000,0.0,-4.011475,-45.280731,-20.660767,69.528549,0.014159,-0.006765
4122979,2022-10-31 00:00:00-04:00,152.110001,153.339996,149.839996,150.779999,1899000.0,0.325,0.0,ZTS,0.0,...,0.0,0.0,0.002155,0.0,-6.076706,-46.958588,-22.192078,67.178452,-0.014220,0.016968
4122980,2022-11-01 00:00:00-04:00,151.399994,154.910004,150.380005,152.720001,2286200.0,0.000,0.0,ZTS,0.0,...,0.0,0.0,0.000000,0.0,0.104279,-42.592453,-22.242554,69.512009,0.012866,-0.010095
4122981,2022-11-02 00:00:00-04:00,152.869995,153.210007,147.190002,147.360001,2638300.0,0.000,0.0,ZTS,0.0,...,0.0,0.0,0.000000,0.0,-3.559326,-49.274902,-25.136749,64.278610,-0.035097,-0.099485


In [11]:
m['Date'] = m['Date'].apply(lambda x : x.split(' ')[0])

In [12]:
m

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Adj Close,...,Book To Market,Cash To Price,Dividend To Price,Earning To Price,30 Day Momentum,180 Day Momentum,360 Day Momentum,1080 Day Momentum,return,er
1,1999-11-19,26.279474,26.317727,24.366850,24.711123,15234146.0,0.000,0.0,A,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.082386,0.090529
2,1999-11-22,25.284907,26.929766,24.519857,26.929766,6577870.0,0.000,0.0,A,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.089783,-0.079438
3,1999-11-23,26.011702,26.700248,24.481602,24.481602,5975611.0,0.000,0.0,A,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.090909,0.017706
4,1999-11-24,24.558100,25.667423,24.481594,25.131887,4843231.0,0.000,0.0,A,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.026562,0.003369
5,1999-11-26,25.017140,25.399665,24.940636,25.208403,1729466.0,0.000,0.0,A,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.003045,0.028967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4122978,2022-10-28,151.777511,153.553736,150.200859,152.955002,1615500.0,0.000,0.0,ZTS,0.0,...,0.0,0.0,0.000000,0.0,-4.011475,-45.280731,-20.660767,69.528549,0.014159,-0.006765
4122979,2022-10-31,152.110001,153.339996,149.839996,150.779999,1899000.0,0.325,0.0,ZTS,0.0,...,0.0,0.0,0.002155,0.0,-6.076706,-46.958588,-22.192078,67.178452,-0.014220,0.016968
4122980,2022-11-01,151.399994,154.910004,150.380005,152.720001,2286200.0,0.000,0.0,ZTS,0.0,...,0.0,0.0,0.000000,0.0,0.104279,-42.592453,-22.242554,69.512009,0.012866,-0.010095
4122981,2022-11-02,152.869995,153.210007,147.190002,147.360001,2638300.0,0.000,0.0,ZTS,0.0,...,0.0,0.0,0.000000,0.0,-3.559326,-49.274902,-25.136749,64.278610,-0.035097,-0.099485


In [13]:
m.to_parquet('data.parquet.gzip',
              compression='gzip')  